## Argo File Loader

In [2]:
import argopy
from argopy import DataFetcher as ArgoDataFetcher
argo_loader=ArgoDataFetcher(src='gdac',ftp="/swot/SUM05/dbalwada/Argo_sync",parallel=True)

import filt_funcs as ff
import gsw

In [3]:
#box=[lon_min,lon_max,lat_min,lat_max,depth_min,depth_max]
box=[60,62,-55,-50,0,2000]

In [18]:
ds = argo_loader.region(box)

In [4]:
def get_box(box,sample_min):
    ds=argo_loader.region(box)
    print('loading points complete')
    
    ds=ds.to_xarray()
    print('to xarray complete')
    
    ds=ds.argo.teos10(['CT','SA','SIG0'])
    ds=ds.argo.point2profile()
    print('point to profile complete')
    
    ds_interp=ff.get_ds_interp(ds,0,2000,sample_min)
    print('interpolation complete')
    
    ds_interp['SPICE'] = gsw.spiciness0(ds_interp.SA,ds_interp.CT).rename('SPICE')
    print('adding spice complete')
        
    return ds_interp

In [5]:
ds=get_box(box,4)

loading points complete
to xarray complete
point to profile complete
interpolation complete
adding spice complete


In [6]:
ds

<xarray.Dataset>
Dimensions:                (N_PROF: 28, PRES_INTERPOLATED: 1000)
Coordinates:
  * N_PROF                 (N_PROF) int64 75 76 77 78 79 ... 239 241 242 243 245
    TIME                   (N_PROF) datetime64[ns] 2020-04-13T22:34:48 ... 20...
    LATITUDE               (N_PROF) float64 -51.8 -51.61 ... -51.12 -51.31
    LONGITUDE              (N_PROF) float64 60.33 61.35 61.53 ... 61.64 61.93
  * PRES_INTERPOLATED      (PRES_INTERPOLATED) int64 0 2 4 6 ... 1994 1996 1998
    N_PROF_NEW             (N_PROF) int64 0 1 2 3 4 5 6 ... 21 22 23 24 25 26 27
Data variables:
    CONFIG_MISSION_NUMBER  (N_PROF) int32 1 1 1 1 1 1 5 ... 4 18 19 21 22 23 25
    CT                     (N_PROF, PRES_INTERPOLATED) float64 5.041 ... 1.589
    CYCLE_NUMBER           (N_PROF) int32 10 11 12 13 14 15 ... 19 21 22 23 25
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'D' 'D' 'D' 'D'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 1901750 1901750 ... 5905366 5905366
    PRES                   (N_PROF, PRES_INTERPOLATED) float32 4.6 ... 1.998e+03
    PSAL                   (N_PROF, PRES_INTERPOLATED) float32 33.78 ... 34.75
    SA                     (N_PROF, PRES_INTERPOLATED) float64 33.94 ... 34.92
    SIG0                   (N_PROF, PRES_INTERPOLATED) float64 26.71 ... 27.81
    TEMP                   (N_PROF, PRES_INTERPOLATED) float32 5.034 ... 1.722
    SPICE                  (N_PROF, PRES_INTERPOLATED) float64 -0.3829 ... -0...
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         /swot/SUM05/dbalwada/Argo_sync
    Fetched_by:           amf2288
    Fetched_date:         2023/05/15
    Fetched_constraints:  [x=60.00/62.00; y=-55.00/-50.00; z=0.0/2000.0]
    Fetched_uri:          /swot/SUM05/dbalwada/Argo_sync/dac/coriolis/1901193...
    history:              Variables filtered according to DATA_MODE; Variable...

In [66]:
ds.to_netcdf("202206/202206_lon:({},{})_lat:({},{}).nc".format(box[0],box[1],box[2],box[3]))

In [23]:
from argopy import IndexFetcher as ArgoIndexFetcher

box_bad=[-148,-147,39,40]#, '2000-01-01', '2021-06'] # need to remove pressure poinds because the region function for this index fetcher seems a bit different.

idx = ArgoIndexFetcher(src='gdac',dataset='phy',mode='standard',
                            ftp="/swot/SUM05/dbalwada/202203-ArgoData").region(box_bad).load()
idx.index

,file,date,latitude,longitude,ocean,profiler_code,institution_code,date_update,wmo,institution,profiler
0,aoml/4900809/profiles/D4900809_001.nc,2007-05-04 13:49:41,39.013,-147.238,P,846,AO,2018-10-15 11:53:32,4900809,"AOML, USA","Webb Research, Seabird sensor"
1,aoml/4900809/profiles/D4900809_002.nc,2007-05-15 03:59:10,39.128,-147.311,P,846,AO,2018-10-15 11:53:32,4900809,"AOML, USA","Webb Research, Seabird sensor"
2,aoml/4900809/profiles/D4900809_003.nc,2007-05-25 20:25:13,39.219,-147.472,P,846,AO,2018-10-15 11:53:33,4900809,"AOML, USA","Webb Research, Seabird sensor"
3,aoml/4900809/profiles/D4900809_004.nc,2007-06-05 13:49:18,39.337,-147.551,P,846,AO,2018-10-15 11:53:33,4900809,"AOML, USA","Webb Research, Seabird sensor"
4,aoml/4900809/profiles/D4900809_005.nc,2007-06-16 09:36:23,39.484,-147.560,P,846,AO,2018-10-15 11:53:33,4900809,"AOML, USA","Webb Research, Seabird sensor"
5,aoml/4900809/profiles/D4900809_006.nc,2007-06-26 23:28:54,39.594,-147.568,P,846,AO,2018-10-15 11:53:33,4900809,"AOML, USA","Webb Research, Seabird sensor"
6,aoml/4900809/profiles/D4900809_007.nc,2007-07-07 20:22:51,39.612,-147.450,P,846,AO,2018-10-15 11:53:34,4900809,"AOML, USA","Webb Research, Seabird sensor"
7,aoml/4900809/profiles/D4900809_008.nc,2007-07-18 09:46:28,39.572,-147.214,P,846,AO,2018-10-15 11:53:34,4900809,"AOML, USA","Webb Research, Seabird sensor"
8,aoml/4900809/profiles/D4900809_009.nc,2007-07-29 06:19:23,39.440,-147.033,P,846,AO,2018-10-15 11:53:34,4900809,"AOML, USA","Webb Research, Seabird sensor"
9,jma/29029/profiles/D29029_322.nc,2006-12-10 06:06:38,39.906,-147.063,P,845,JA,2015-06-21 03:45:18,29029,"JMA, Japan","Webb Research, no conductivity"
